## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,CityID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Salalah,OM,17.0151,54.0924,21.20,70,2,0.68,clear sky
1,1,Staritsa,RU,56.5075,34.9354,-3.58,88,90,5.76,overcast clouds
2,2,Pevek,RU,69.7008,170.3133,-30.58,98,100,4.22,overcast clouds
3,3,Colon,PA,9.3592,-79.9014,34.45,64,14,5.57,few clouds
4,4,Tasiilaq,GL,65.6145,-37.6368,-14.74,66,100,1.81,overcast clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is your desired minimum temperature for your trip? "))
max_temp = float(input("What is your desired maximum temperature for your trip? "))

What is your desired minimum temperature for your trip? 5
What is your desired maximum temperature for your trip? 25


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) &
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head()

,CityID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Salalah,OM,17.0151,54.0924,21.20,70,2,0.68,clear sky
5,5,Rocha,UY,-34.4833,-54.3333,18.90,94,100,8.10,overcast clouds
6,6,Bluff,NZ,-46.6000,168.3333,11.00,70,100,5.79,overcast clouds
11,11,Irbil,IQ,36.1926,44.0106,6.90,61,0,1.54,clear sky
14,14,Kununurra,AU,-15.7667,128.7333,24.99,100,67,0.00,broken clouds


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().sum()

CityID                 0
City                   0
Country                6
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()
clean_df

,CityID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Salalah,OM,17.0151,54.0924,21.20,70,2,0.68,clear sky
5,5,Rocha,UY,-34.4833,-54.3333,18.90,94,100,8.10,overcast clouds
6,6,Bluff,NZ,-46.6000,168.3333,11.00,70,100,5.79,overcast clouds
11,11,Irbil,IQ,36.1926,44.0106,6.90,61,0,1.54,clear sky
14,14,Kununurra,AU,-15.7667,128.7333,24.99,100,67,0.00,broken clouds
...,...,...,...,...,...,...,...,...,...,...
673,673,West Wendover,US,40.7391,-114.0733,7.22,48,0,1.25,clear sky
674,674,Newnan,US,33.3807,-84.7997,21.75,34,0,2.57,clear sky
677,677,Hue,VN,16.4667,107.6000,22.06,94,20,0.51,few clouds
679,679,Dakoro,NE,14.5106,6.7650,20.84,10,3,4.59,clear sky


In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Salalah,OM,21.20,clear sky,17.0151,54.0924,
5,Rocha,UY,18.90,overcast clouds,-34.4833,-54.3333,
6,Bluff,NZ,11.00,overcast clouds,-46.6000,168.3333,
11,Irbil,IQ,6.90,clear sky,36.1926,44.0106,
14,Kununurra,AU,24.99,broken clouds,-15.7667,128.7333,
17,Ponta Do Sol,PT,17.35,scattered clouds,32.6667,-17.1000,
18,Cape Town,ZA,21.72,clear sky,-33.9258,18.4232,
24,Port Elizabeth,ZA,23.16,overcast clouds,-33.9180,25.5701,
25,Kruisfontein,ZA,22.28,overcast clouds,-34.0033,24.7314,
31,Birao,CF,23.20,scattered clouds,10.2849,22.7882,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"   
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
        
    except (IndexError):
        print("Hotel not found...Skipping.")
        

Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.
Hotel not found...Skipping.


In [9]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.loc[(hotel_df["Hotel Name"]!='')]
clean_hotel_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Salalah,OM,21.20,clear sky,17.0151,54.0924,"HAMDAN PLAZA HOTEL SALALAH, an HTG Hotel"
5,Rocha,UY,18.90,overcast clouds,-34.4833,-54.3333,Hotel Arrarte
6,Bluff,NZ,11.00,overcast clouds,-46.6000,168.3333,Bluff Homestead - Guesthouse & Campervan Park
11,Irbil,IQ,6.90,clear sky,36.1926,44.0106,Erbil International Hotel
14,Kununurra,AU,24.99,broken clouds,-15.7667,128.7333,Hotel Kununurra


In [10]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"

# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="CityID")

In [11]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °C</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [12]:
# 11a. Add a marker layer for each city to the map. 
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))

In [13]:
jupyter nbextension enable --py gmaps
jupyter nbextension enable --py widgetsnbextension

SyntaxError: invalid syntax (Temp/ipykernel_16100/2514220349.py, line 1)